In [1]:
from dotenv import load_dotenv
from langchain_teddynote import logging

# .env 파일 로드
load_dotenv()

# langSmith에 로깅 할 프로젝트 명을 입력
logging.langsmith("LANGCHAIN-CHOON")
# logging.langsmith("LANGCHAIN-CHOON", set_enable=False)      # LangSmith 추적 비활성화

LangSmith 추적을 시작합니다.
[프로젝트명]
LANGCHAIN-CHOON


## 데이터를 효과적으로 전달하는 방법

- RunnablePassthrough : 입력을 변경하지 않거나 추가 키를 더하여 전달
- RunnablePassthrough() : 단독으로 호출되면, 단순히 입력을 받아 그대로 전달

### RunnablePassthrough

In [2]:
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI

prompt = PromptTemplate.from_template("{num} 의 10배는?")
llm = ChatOpenAI(model="gpt-4o")

chain = prompt | llm

chain을 `invoke()` 하여 실행할 때는 입력 데이터의 타입은 딕셔너리여야 한다

In [3]:
chain.invoke("{num}: 10")

AIMessage(content='10의 10배는 100입니다.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 11, 'prompt_tokens': 18, 'total_tokens': 29, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-2024-08-06', 'system_fingerprint': 'fp_4691090a87', 'finish_reason': 'stop', 'logprobs': None}, id='run-ebc189d6-aa53-49d8-864f-096946e48f47-0', usage_metadata={'input_tokens': 18, 'output_tokens': 11, 'total_tokens': 29, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

`RunnablePassthrough` 는 `runnable` 객체이며, `runnable` 객체는 `invoke()` 메서드를 사용하여 별도로 실행이 가능하다

In [5]:
from langchain_core.runnables import RunnablePassthrough

RunnablePassthrough().invoke("num: 10")

'num: 10'

아래와 같이 RunnablePassthrough 를 체인에 적용하여 사용할 수 있다

In [6]:
runnable_chain = {"num": RunnablePassthrough()} | prompt | llm

# dict 값이 RunnablePassthrough() 로 변경
runnable_chain.invoke(19)

AIMessage(content='19의 10배는 190입니다.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 11, 'prompt_tokens': 14, 'total_tokens': 25, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-2024-08-06', 'system_fingerprint': 'fp_50cad350e4', 'finish_reason': 'stop', 'logprobs': None}, id='run-fe5fc4c7-e1de-4278-aa81-695e32e5e445-0', usage_metadata={'input_tokens': 14, 'output_tokens': 11, 'total_tokens': 25, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

### RunnableParallel

Runnable 객체를 병렬로 실행하기 위해 사용

In [ ]:
from langchain_core.runnables import RunnableParallel

chain1 = (
    {"country": RunnablePassthrough()}
    | PromptTemplate.from_template("{country} 의 수도는 어떻게 되나요?")
    | ChatOpenAI(model="gpt-4o")
)

chain2 = (
    {"country": RunnablePassthrough()}
    | PromptTemplate.from_template("{country} 의 면적은 어떻게 되나요?")
    | ChatOpenAI(model="gpt-4o")
)

combined_chain = RunnableParallel(capital=chain1, area=chain2)
combined_chain.invoke("미국")

{'capital': AIMessage(content='미국의 수도는 워싱턴 D.C.입니다.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 14, 'prompt_tokens': 16, 'total_tokens': 30, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-2024-08-06', 'system_fingerprint': 'fp_50cad350e4', 'finish_reason': 'stop', 'logprobs': None}, id='run-092409be-8600-49fc-9a7a-34311caa53b8-0', usage_metadata={'input_tokens': 16, 'output_tokens': 14, 'total_tokens': 30, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}}),
 'area': AIMessage(content='미국의 총 면적은 약 983만 평방킬로미터(약 380만 평방마일)입니다. 이는 세계에서 세 번째로 큰 나라로, 러시아와 캐나다에 이어지는 면적입니다.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 52, 'prompt_tokens': 17, 'total_tokens': 

### RunnableLambda

RunnableLambda를 사용하여 사용자 정의 함수를 맵핑할 수 있다

In [38]:
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_openai import ChatOpenAI
from datetime import datetime


def get_today(a):
    print(a)
    return datetime.today().strftime("%b-%d")


# get_today(None)

In [39]:
from langchain_core.runnables import RunnableLambda, RunnablePassthrough
from operator import itemgetter

prompt = PromptTemplate.from_template(
    "{today}가 생일인 유명인 {n} 명을 나열하세요. 생년월일을 표기해 주세요."
)
llm = ChatOpenAI(model="gpt-4o")

chain = (
    {"today": RunnableLambda(get_today), "n": itemgetter("n")}
    | prompt
    | llm
    | StrOutputParser()
)

In [40]:
print(chain.invoke({"n": 3}))

{'n': 3}
다음은 2월 1일에 태어난 유명인 세 명입니다:

1. 해리 스타일스 (Harry Styles) - 1994년 2월 1일
2. 리카르도 몬탈반 (Ricardo Montalbán) - 1920년 2월 1일
3. 랭스 암스트롱 (Langston Hughes) - 1902년 2월 1일

이 외에도 많은 유명인들이 이 날 태어났습니다.
